### Dynamic programming

In [65]:
def assert_sol(K, items, opt_value, sol):
    v = 0
    w = 0
    for i in range(len(sol)):
        if sol[i] == 1:
            v += items[i].value
            w += items[i].weight
    assert(v == opt_value)
    assert(w <= K)

In [66]:
def dynamic_prog(K, items):
    N = len(items)
    tab = [[0 for i in range(N + 1)] for j in range(K + 1)]
    for i in range(1, K + 1):
        for j in range(1, N + 1):
            if items[j - 1].weight > i:
                tab[i][j] = tab[i][j - 1]
            else:                
                tab[i][j] = max(tab[i][j - 1], tab[i - items[j - 1].weight][j - 1] + items[j - 1].value)
                
    opt = tab[K][N]
    taken = [0] * N
    i, j2 = (K, N)
    while j2 >= 1:
        if tab[i][j2] == tab[i][j2 - 1]:
            taken[j2 - 1] = 0
        else:
            taken[j2 - 1] = 1
            i -= items[j2 - 1].weight
        j2 -= 1
    assert_sol(K, items, opt, taken)
    return opt, taken, tab

#### Run the values

In [67]:
from collections import namedtuple
Item = namedtuple("Item", ['index', 'value', 'weight'])
K = 10 
capacity = K
items = [Item(index=0, value=5, weight=4), Item(index=1, value=6, weight=5), Item(index=2, value=3, weight=2)]

dynamic_prog(K, items)

(11,
 [1, 1, 0],
 [[0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 3],
  [0, 0, 0, 3],
  [0, 5, 5, 5],
  [0, 5, 6, 6],
  [0, 5, 6, 8],
  [0, 5, 6, 9],
  [0, 5, 6, 9],
  [0, 5, 11, 11],
  [0, 5, 11, 11]])

### Conventional way

In [68]:
    # a trivial algorithm for filling the knapsack
    # it takes items in-order until the knapsack is full
value = 0
weight = 0
taken = [0]*len(items)

for item in items:
        if weight + item.weight <= capacity:
            taken[item.index] = 1
            value += item.value
            weight += item.weight
    
# prepare the solution in the specified output format
output_data = str(value) + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, taken))
print(output_data)

11 0
1 1 0


### Linprog way

In [69]:
value = []
weight = []

for item in items:
    value.append(item.value)
    weight.append(item.weight)
print(value,weight)

[5, 6, 3] [4, 5, 2]


In [17]:
import numpy as np

# Set of items
I = set(range(1, 4))

# Weight associated with each item
w = dict(zip(I, weight))

# Value associated with each item
v = dict(zip(I, value))

In [28]:
print(w,v,K)

{1: 4, 2: 5, 3: 2} {1: 5, 2: 6, 3: 3} 10


In [26]:
# Costs
c = -np.array(list(v.values()))

# Inequality constraints matrix
A_ub = np.array([
    np.array(list(w.values()))
])

# Upper bounds for linear inequality constraints
b_ub = np.array([K])

# Bounds (one quantity of each item)
bounds = [(0, 1),] * len(v)

In [31]:
#print(A_ub, b_ub, bounds)

In [27]:
from scipy.optimize import linprog

# Obtain solution
sol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

print(sol)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -12.8
              x: [ 1.000e+00  8.000e-01  1.000e+00]
            nit: 1
          lower:  residual: [ 1.000e+00  8.000e-01  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [ 0.000e+00  2.000e-01  0.000e+00]
                 marginals: [-2.000e-01  0.000e+00 -6.000e-01]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-1.200e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


### Search method

In [46]:
def DFSearch(K, items):
    n1 = Node(K, items, [])

    items2 = sorted(items, key=lambda x: x.value/x.weight, reverse=True)
    best_node, visited = search(K, items2, [n1])
    sol = [0] * len(best_node.sol)
    for i in range(len(best_node.sol)):
        sol[items2[i].index] = best_node.sol[i]
    assert_sol(K, items, best_node.value, sol)
    return best_node.value, sol, visited

In [37]:
class Node():
    def __init__(self, K, items, sol):
        w, v = (0, 0)
        self.is_leaf = (len(sol) == len(items))
        for n in range(len(sol)):
            if sol[n]:
                w += items[n].weight
                v += items[n].value
        if w > K:
            self.feasible = False
        else:
            self.value = v
            self.feasible = True
            self.sol = sol
        if self.feasible:
            if not self.is_leaf:
                self.estimate = relax(K, items, sol, self.value)
            else:
                self.estimate = self.value

In [39]:
def relax(K, items, curr_sol, curr_value):
    global SORTED_RATIO
    ratio = SORTED_RATIO[len(curr_sol):len(items)]
    v = curr_value
    w = 0
    for i in range(len(curr_sol)):
        if curr_sol[i] == 1:
            w += items[i].weight
    for r in ratio:
        w += items[r[1]].weight
        if w <= K:
            v += items[r[1]].value
        else:
            extra = K -w
            perc = 1 - extra/items[r[1]].weight
            v += perc * items[r[1]].value
            return v
    return v

In [33]:
def search(K, items, node_list):
    curr_best = None
    visited = 0
    time_limit = 300 # seconds
    start_time = time.time()
    while len(node_list) > 0:
        node = node_list.pop()
        if node.feasible:
            if not node.is_leaf:
                if curr_best is None or node.estimate > curr_best.value:
                    visit(node, node_list, K, items)
                    visited += 1
            else:
                if curr_best is None or node.value > curr_best.value:
                    curr_best = node
        if time.time() - start_time > time_limit:
            print("Time interruption")
            break
    return curr_best, visited

In [50]:
def visit(node, node_list, K, items):
    sol1 = node.sol.copy()
    sol_l = sol1 + [1]
    sol_r = sol1 + [0]
    left = Node(K, items, sol_l)
    right = Node(K, items, sol_r)
    #depth_first(node_list, left, right)
    #best_first(node_list, left, right)
    rand_first(node_list, left, right)

In [35]:
def depth_first(node_list, left, right):
    if right.feasible:
        node_list.append(right)
    if left.feasible:
        node_list.append(left)


def best_first(node_list, left, right):
    if right.feasible:
        node_list.append(right)
    if left.feasible:
        node_list.append(left)
    if left.feasible or right.feasible:
        node_list.sort(key=lambda x: x.value)


def rand_first(node_list, left, right):
    r = random.random()
    if r > 0.5:
        l = [left, right]
    else:
        l = [right, left]
    if l[0].feasible:
        node_list.append(l[0])
    if l[1].feasible:
        node_list.append(l[1])

In [43]:
def make_sorted_ratio(items):
    ratio = [(0, 0)] * len(items)
    for idx in range(len(items)):
        ratio[idx] = (items[idx].value /
                     items[idx].weight, idx)
    ratio.sort(key=lambda x: x[0])
    return ratio

In [52]:
import time
import random

SORTED_RATIO = make_sorted_ratio(items)

DFSearch(K, items)

(9, [0, 1, 1], 4)

### Ortools

In [58]:
from ortools.sat.python import cp_model

In [70]:
# Instantiate model and solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# 1. Variables
capacity = K
weight1 = model.NewIntVar(0, 1, 'weight1')
weight2 = model.NewIntVar(0, 1, 'weight2')
weight3 = model.NewIntVar(0, 1, 'weight3')

In [71]:
# 2. Constraints
model.Add(4 * weight1 + 5 * weight2 + 2 * weight3 <= capacity)

In [72]:
# 3. Objective
model.Maximize(5  * weight1 + 6 * weight2 + 3 * weight3)

In [73]:
# Solve problem
status = solver.Solve(model)

# If an optimal solution has been found, print results
if status == cp_model.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.WallTime():.2f} milliseconds')
    print()
    print(f'Optimal value = {5*solver.Value(weight1) + 6*solver.Value(weight2) + 3*solver.Value(weight3)} popularity')
    print('Food:')
    print(f' - Weight1 = {solver.Value(weight1)}')
    print(f' - Weight2  = {solver.Value(weight2)}')
    print(f' - Weight3  = {solver.Value(weight3)}')
else:
    print('The solver could not find an optimal solution.')

================= Solution =================
Solved in 0.00 milliseconds

Optimal value = 11 popularity
Food:
 - Weight1 = 1
 - Weight2  = 1
 - Weight3  = 0
